# __Hashtags Validation__

In [38]:
import pandas as pd
from IPython.display import clear_output, display
import os, datetime
import numpy as np

In [2]:
DATA_PATH = 'data/tweets/'

week_list = [week_dir for week_dir in os.listdir(DATA_PATH) if os.path.isdir(DATA_PATH+week_dir) and not week_dir.endswith('.ipynb_checkpoints')]
print('Amount of extracted weeks: ', len(week_list))

Amount of extracted weeks:  26


In [3]:
week_list.sort()
week_list.remove('week_pr_01')
week_list.insert(week_list.index('week_12')+1, 'week_pr_01')
week_list.remove('week_pr_02')
week_list.insert(week_list.index('week_pr_01')+1, 'week_pr_02')
week_list.remove('week_pr_03')
week_list.insert(week_list.index('week_17')+1, 'week_pr_03')
week_list.remove('week_pr_04')
week_list.insert(week_list.index('week_21')+1, 'week_pr_04')

print(week_list)

['week_01', 'week_02', 'week_03', 'week_04', 'week_05', 'week_06', 'week_07', 'week_08', 'week_09', 'week_10', 'week_11', 'week_12', 'week_pr_01', 'week_pr_02', 'week_13', 'week_14', 'week_15', 'week_16', 'week_17', 'week_pr_03', 'week_18', 'week_19', 'week_20', 'week_21', 'week_pr_04', 'week_22']


In [4]:
user_list = []
hashtag_list = []

In [17]:
for week in week_list:
    clear_output(wait=True)
    print(f'Starting {week}:')
    week_df = pd.read_parquet(f'{DATA_PATH+week}/merged_{week}.parquet')
    for idx, tweet in week_df.iterrows():
        #usr_name = eval(tweet['user'])['username']
        hashtags = eval(tweet['hashtags'])
        #if usr_name not in user_list:
        #    user_list.append(usr_name)
        if hashtags:
            for hashtag in hashtags:
                hashtag_list.append(hashtag.lower())
                #hasht_low = hashtag.lower()
                #if hasht_low not in hashtag_list:
                #    hashtag_list.append(hasht_low)
    print(f'''
    Finished {week}:
    Total number of users: {len(user_list)}
    Total number of hashtags: {len(hashtag_list)}
    \n
    ''')

Starting week_22:

    Finished week_22:
    Total number of users: 472852
    Total number of hashtags: 5189068
    

    


In [25]:
user_series = pd.Series(user_list)
hashtag_series = pd.Series(hashtag_list)

In [26]:
user_series.drop_duplicates(inplace=True)
len(user_series)

472852

In [27]:
user_series[:20]

0         Indignado72
1     EscutaZeNinguem
2       OSul_noticias
3          Alvesalces
4             guinody
5      ANA_PAULA_FONS
6         libolsonaro
7         Sandroka131
8           gomessamg
9           teddyfps1
10         thivagando
11    AndreaPorto2021
12          thetunnes
13    ngelomarchioll1
14     MarceloAOrtega
15     eriicksonsilva
16            Elmosol
17     sbsergiobastos
18         silviacbap
19     LuizzPatriotaa
dtype: object

In [28]:
len(hashtag_series)

5189068

In [40]:
hashtag_series.value_counts()[:30]

cpidacovid                     609488
forabolsonaro                  595474
forabolsonarogenocida          150814
cpidapandemia                  143894
renanvagabundo                 116162
cpidocirco                     111937
renansabiadetudo                88980
29mforabolsonaro                79676
euautorizopresidente            79206
bolsonarogenocida               57439
cpidotse                        56181
bolsonaropresidenteate2026      55712
lulaladrao                      55328
3jforabolsonaro                 52604
barrosonacadeia                 51438
cpidogenocidio                  50534
euapoiovotoauditavel            48931
covid19                         43422
cpidacovid19                    41100
aglobonaopresta                 39479
19jforabolsonaro                38188
impeachmentbolsonarourgente     36020
bolsonarovaicair                34391
24jforabolsonaro                33803
lulalivrebrasillivre            33762
euapoiobolsonaro2022            31651
brasil      

In [62]:
top_30 = list(hashtag_series.value_counts()[:30].index)

In [42]:
user_hash = {}
for user in user_list:
    user_hash[user] = np.zeros(30, dtype=int)

In [55]:
for week in week_list:
    clear_output(wait=True)
    print(f'Current week: {week}...')
    week_df = pd.read_parquet(f'{DATA_PATH+week}/merged_{week}.parquet')
    for idx, tweet in week_df.iterrows():
        hashtags = eval(tweet['hashtags'])
        if hashtags:
            usr_name = eval(tweet['user'])['username']
            for hashtag in hashtags:
                if hashtag in top_30:
                    idx = top_30.index(hashtag)
                    user_hash[usr_name][idx] = 1
    print(f'Finished week: {week}')
    

Current week: week_22...
Finished week: week_22


In [63]:
columns = ['user']+top_30
columns

['user',
 'cpidacovid',
 'forabolsonaro',
 'forabolsonarogenocida',
 'cpidapandemia',
 'renanvagabundo',
 'cpidocirco',
 'renansabiadetudo',
 '29mforabolsonaro',
 'euautorizopresidente',
 'bolsonarogenocida',
 'cpidotse',
 'bolsonaropresidenteate2026',
 'lulaladrao',
 '3jforabolsonaro',
 'barrosonacadeia',
 'cpidogenocidio',
 'euapoiovotoauditavel',
 'covid19',
 'cpidacovid19',
 'aglobonaopresta',
 '19jforabolsonaro',
 'impeachmentbolsonarourgente',
 'bolsonarovaicair',
 '24jforabolsonaro',
 'lulalivrebrasillivre',
 'euapoiobolsonaro2022',
 'brasil',
 'votoimpressoauditavelja',
 'brasilpelovotoauditavel',
 'dia07vaisergigante']

In [68]:
user_hash_matrix = []

for key in list(user_hash.keys()):
    user_h = [key]+list(user_hash[key])
    user_hash_matrix.append(user_h)


In [69]:
usr_hash_df = pd.DataFrame(columns=columns, data=user_hash_matrix)

In [70]:
usr_hash_df.shape

(472852, 31)

In [71]:
usr_hash_df.head()

,user,cpidacovid,forabolsonaro,forabolsonarogenocida,cpidapandemia,renanvagabundo,cpidocirco,renansabiadetudo,29mforabolsonaro,euautorizopresidente,...,19jforabolsonaro,impeachmentbolsonarourgente,bolsonarovaicair,24jforabolsonaro,lulalivrebrasillivre,euapoiobolsonaro2022,brasil,votoimpressoauditavelja,brasilpelovotoauditavel,dia07vaisergigante
0,Indignado72,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,EscutaZeNinguem,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,OSul_noticias,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Alvesalces,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,guinody,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
usr_hash_df.to_parquet('data/user_hashtag_matrix.parquet', index=False)

In [89]:
# neutral hashtags sets
cpidacovid = set(usr_hash_df[usr_hash_df['cpidacovid']==1]['user'])
cpidapandemia = set(usr_hash_df[usr_hash_df['cpidapandemia']==1]['user'])
covid19 = set(usr_hash_df[usr_hash_df['covid19']==1]['user'])
cpidacovid19 = set(usr_hash_df[usr_hash_df['cpidacovid19']==1]['user'])
brasil = set(usr_hash_df[usr_hash_df['brasil']==1]['user'])

# pro government hashtags sets
renanvagabundo = set(usr_hash_df[usr_hash_df['renanvagabundo']==1]['user'])
cpidocirco = set(usr_hash_df[usr_hash_df['cpidocirco']==1]['user'])
renansabiadetudo = set(usr_hash_df[usr_hash_df['renansabiadetudo']==1]['user'])
euautorizopresidente = set(usr_hash_df[usr_hash_df['euautorizopresidente']==1]['user'])
cpidotse = set(usr_hash_df[usr_hash_df['cpidotse']==1]['user'])
bolsonaropresidenteate2026 = set(usr_hash_df[usr_hash_df['bolsonaropresidenteate2026']==1]['user'])

# against government hashtags sets
forabolsonaro = set(usr_hash_df[usr_hash_df['forabolsonaro']==1]['user'])
forabolsonarogenocida = set(usr_hash_df[usr_hash_df['forabolsonarogenocida']==1]['user'])
_29mforabolsonaro = set(usr_hash_df[usr_hash_df['29mforabolsonaro']==1]['user'])
bolsonarogenocida = set(usr_hash_df[usr_hash_df['bolsonarogenocida']==1]['user'])
_3jforabolsonaro = set(usr_hash_df[usr_hash_df['3jforabolsonaro']==1]['user'])
cpidogenocidio = set(usr_hash_df[usr_hash_df['cpidogenocidio']==1]['user'])

In [94]:
print(f'''
Neutral Hashtags

#cpidacovid: {len(cpidacovid)}
#cpidapandemia: {len(cpidapandemia)}
#covid19: {len(covid19)}
#cpidacovid19: {len(cpidacovid19)}
#brasil: {len(brasil)}

Pro Government Hashtags

#renanvagabundo: {len(renanvagabundo)}
#cpidocirco: {len(cpidocirco)}
#renansabiadetudo: {len(renansabiadetudo)}
#euautorizopresidente: {len(euautorizopresidente)}
#cpidotse: {len(cpidotse)}
#bolsonaropresidenteate2026: {len(bolsonaropresidenteate2026)}

Against Government Hastags

#forabolsonaro: {len(forabolsonaro)}
#forabolsonarogenocida: {len(forabolsonarogenocida)}
#29mforabolsonaro: {len(_29mforabolsonaro)}
#bolsonarogenocida: {len(bolsonarogenocida)}
#3jforabolsonaro: {len(_3jforabolsonaro)}
#cpidogenocidio: {len(cpidogenocidio)}
''')


Neutral Hashtags

#cpidacovid: 4181
#cpidapandemia: 995
#covid19: 2107
#cpidacovid19: 397
#brasil: 3677

Pro Government Hashtags

#renanvagabundo: 414
#cpidocirco: 764
#renansabiadetudo: 148
#euautorizopresidente: 141
#cpidotse: 113
#bolsonaropresidenteate2026: 27

Against Government Hastags

#forabolsonaro: 21780
#forabolsonarogenocida: 2013
#29mforabolsonaro: 267
#bolsonarogenocida: 1578
#3jforabolsonaro: 343
#cpidogenocidio: 163



In [126]:
print(f'''
Neutral
#cpidacovid ({len(cpidacovid)}) x #covid19 ({len(covid19)}) : {len(cpidacovid.intersection(covid19))}
#cpidacovid ({len(cpidacovid)}) x #cpidapandemia ({len(cpidapandemia)}): {len(cpidacovid.intersection(cpidapandemia))}
#cpidacovid ({len(cpidacovid)}) x #brasil ({len(brasil)}): {len(cpidacovid.intersection(brasil))}

Against CPI
#cpidocirco ({len(cpidocirco)}) x #renanvagabundo ({len(renanvagabundo)}): {len(cpidocirco.intersection(renanvagabundo))}
#cpidocirco ({len(cpidocirco)}) x #renansabiadetudo ({len(renansabiadetudo)}): {len(cpidocirco.intersection(renansabiadetudo))}
#cpidocirco ({len(cpidocirco)}) x #euautorizopresidente ({len(euautorizopresidente)}): {len(cpidocirco.intersection(euautorizopresidente))}
#cpidocirco ({len(cpidocirco)}) x #bolsonaropresidenteate2026 ({len(bolsonaropresidenteate2026)}): {len(cpidocirco.intersection(bolsonaropresidenteate2026))}

Pro CPI
#forabolsonaro ({len(forabolsonaro)}) x #forabolsonarogenocida ({len(forabolsonarogenocida)}): {len(forabolsonaro.intersection(forabolsonarogenocida))}
#forabolsonaro ({len(forabolsonaro)}) x #bolsonarogenocida ({len(bolsonarogenocida)}): {len(forabolsonaro.intersection(bolsonarogenocida))}
#forabolsonaro ({len(forabolsonaro)}) x #cpidogenocidio ({len(cpidogenocidio)}): {len(forabolsonaro.intersection(cpidogenocidio))}
#forabolsonarogenocida ({len(forabolsonarogenocida)}) x #bolsonarogenocida ({len(bolsonarogenocida)}): {len(forabolsonarogenocida.intersection(bolsonarogenocida))}
''')


Neutral
#cpidacovid (4181) x #covid19 (2107) : 187
#cpidacovid (4181) x #cpidapandemia (995): 454
#cpidacovid (4181) x #brasil (3677): 197

Against CPI
#cpidocirco (764) x #renanvagabundo (414): 52
#cpidocirco (764) x #renansabiadetudo (148): 15
#cpidocirco (764) x #euautorizopresidente (141): 11
#cpidocirco (764) x #bolsonaropresidenteate2026 (27): 7

Pro CPI
#forabolsonaro (21780) x #forabolsonarogenocida (2013): 1163
#forabolsonaro (21780) x #bolsonarogenocida (1578): 789
#forabolsonaro (21780) x #cpidogenocidio (163): 45
#forabolsonarogenocida (2013) x #bolsonarogenocida (1578): 204



In [133]:
print(f'''
Pro x Against
#forabolsonaro ({len(forabolsonaro)}) x #cpidocirco ({len(cpidocirco)}): {len(forabolsonaro.intersection(cpidocirco))}
#forabolsonaro ({len(forabolsonaro)}) x #renanvagabundo ({len(cpidocirco)}): {len(forabolsonaro.intersection(renanvagabundo))}
#forabolsonaro ({len(forabolsonaro)}) x #euautorizopresidente ({len(euautorizopresidente)}): {len(forabolsonaro.intersection(euautorizopresidente))}
#bolsonarogenocida ({len(bolsonarogenocida)}) x #cpidocirco ({len(cpidocirco)}): {len(bolsonarogenocida.intersection(cpidocirco))}
#bolsonarogenocida ({len(bolsonarogenocida)}) x #renanvagabundo({len(renanvagabundo)}): {len(bolsonarogenocida.intersection(renanvagabundo))}
#cpidogenocidio ({len(cpidogenocidio)}) x #cpidocirco({len(cpidocirco)}): {len(cpidogenocidio.intersection(cpidocirco))}
''')


Pro x Against
#forabolsonaro (21780) x #cpidocirco (764): 11
#forabolsonaro (21780) x #renanvagabundo (764): 3
#forabolsonaro (21780) x #euautorizopresidente (141): 2
#bolsonarogenocida (1578) x #cpidocirco (764): 5
#bolsonarogenocida (1578) x #renanvagabundo(414): 2
#cpidogenocidio (163) x #cpidocirco(764): 3



In [140]:
print(f'''
Neutral x Against and Pro
#cpidacovid ({len(cpidacovid)}) x #cpidogenocidio ({len(cpidogenocidio)}): {len(cpidacovid.intersection(cpidogenocidio))}
#cpidacovid ({len(cpidacovid)}) x #cpidocirco ({len(cpidocirco)}): {len(cpidacovid.intersection(cpidocirco))}
#brasil ({len(brasil)}) x #forabolsonaro ({len(forabolsonaro)}): {len(brasil.intersection(forabolsonaro))}
#brasil ({len(brasil)}) x #cpidocirco ({len(cpidocirco)}): {len(brasil.intersection(cpidocirco))}
#covid19 ({len(covid19)}) x #forabolsonaro ({len(forabolsonaro)}): {len(covid19.intersection(forabolsonaro))}
#covid19 ({len(covid19)}) x #cpidocirco ({len(cpidocirco)}): {len(covid19.intersection(cpidocirco))}
''')


Neutral x Against and Pro
#cpidacovid (4181) x #cpidogenocidio (163): 83
#cpidacovid (4181) x #cpidocirco (764): 43
#brasil (3677) x #forabolsonaro (21780): 436
#brasil (3677) x #cpidocirco (764): 25
#covid19 (2107) x #forabolsonaro (21780): 252
#covid19 (2107) x #cpidocirco (764): 14

